In [19]:
import numpy as np
import pandas as pd
import os
from collections import OrderedDict

In [20]:
from platform import python_version
print(python_version())

3.6.6


### Exp of classifing movie genres based on their events.


In [21]:
movies = pd.read_csv('../../data/movie_metadata_cleaned.tsv', sep = '\t')

In [22]:
movies.head(2)

,Wikipedia_id,Freebase_id,Name,Release_date,Revenue,Runtime,Languages,Countries,Genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,dict_values(['English Language']),dict_values(['United States of America']),Thriller
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,dict_values(['English Language']),dict_values(['United States of America']),Drama


### Read in events data

In [23]:
event_file = [item for item in os.listdir('../../data/protag_events_v2/') if 'txt' in item]

In [24]:
d = {}
for name in event_file:
    with open(os.path.join('../../data/protag_events_v2/', name), 'r') as f:
        #only use the verbs
        verbs = []
        for line in [line.strip() for line in f.readlines()]:
            line = line.split(' ')
            verbs.append(line[1])
        if len(verbs) > 1:    
            d[int(name.split('_')[0])] = verbs

In [25]:
d = {}
for name in event_file:
    with open(os.path.join('../../data/protag_events_v2/', name), 'r') as f:
        #only use the verbs
        nlines = []
        for line in [line.strip() for line in f.readlines()]:
            line = line.split(' ')
            line[0] = 'protagonist'
            line = ' '.join([item for item in line[0:2]])
            nlines.append(line)
        if len(nlines) > 1:    
            d[int(name.split('_')[0])] = nlines

In [26]:
movies.Wikipedia_id = movies.Wikipedia_id.astype(int)

In [27]:
movies['Events'] = movies['Wikipedia_id'].map(d)

In [28]:
movies = movies.dropna(subset = ['Events'])

In [29]:
movies['Genre_code'] = movies.Genres.astype("category").cat.codes

In [37]:
 movies.Genres.astype("category").cat.categories

Index(['Action', 'Comedy', 'Drama', 'Romance Film', 'Thriller'], dtype='object')

In [41]:
dict( enumerate(movies.Genres.astype("category").cat.categories) )

{0: 'Action', 1: 'Comedy', 2: 'Drama', 3: 'Romance Film', 4: 'Thriller'}

In [30]:
movies.head(2)

,Wikipedia_id,Freebase_id,Name,Release_date,Revenue,Runtime,Languages,Countries,Genres,Events,Genre_code
13,11250635,/m/02r52hc,The Mechanical Monsters,NaN,NaN,NaN,dict_values(['English Language']),dict_values(['United States of America']),Action,"[protagonist weather-57, protagonist battle-36...",0
81,164387,/m/015wmg,...And Justice for All,1979-06-29,33300000.0,120.0,dict_values(['English Language']),dict_values(['United States of America']),Thriller,"[protagonist exist-47.1-1-1, protagonist conti...",4


In [31]:
movies.Genre_code.value_counts()

2    2421
4    1637
1    1229
3     669
0     494
Name: Genre_code, dtype: int64

In [32]:
movies.to_csv('../../data/metadata_events_table.tsv', sep = '\t', index=False)